In [43]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1. Data Preparation

# Loading the dataset
df = pd.read_csv('dataset.csv')
df = df[df['BurnRate'].str.len() > 2] #ignoring all BurnRates with empty lists

# Converting the BurnRate string representation of lists into actual lists
df['BurnRate'] = df['BurnRate'].apply(lambda x: [float(i) for i in x.strip('[]').split(',')])


# Ensure all lists have the same length, if not, pad with zeros (or other appropriate value)
max_length = df['BurnRate'].apply(len).max()
df['BurnRate'] = df['BurnRate'].apply(lambda x: x + [0] * (max_length - len(x)))

# Splitting the dataset into training and test sets
X = df.drop(columns='BurnRate').values
Y = np.stack(df['BurnRate'].values)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X)
# 2. Model Building

model = Sequential()

# Add LSTM layer - you can adjust the number of units and return_sequences parameter as needed
model.add(LSTM(50, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(max_length))

model.compile(optimizer='adam', loss='mean_squared_error')

# 3. Training & Evaluation

# Reshaping X data to fit LSTM 3D input (samples, timesteps, features)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

[[ 2.00000000e+00  2.00000000e+00  1.00000000e+00 ...  4.00000000e+00
   4.71238898e+00  7.85398163e+00]
 [ 2.00000000e+00  2.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   4.71238898e+00  7.85398163e+00]
 [ 2.00000000e+00  2.00000000e+00  1.00000000e+00 ...  4.00000000e+00
   4.71238898e+00  7.85398163e+00]
 ...
 [ 5.20000000e+01  6.00000000e+00  6.00000000e+00 ...  4.00000000e+00
  -2.94053072e+04  3.23458380e+04]
 [ 5.20000000e+01  6.00000000e+00  6.00000000e+00 ...  1.00000000e+00
  -2.94053072e+04  3.23458380e+04]
 [ 5.20000000e+01  6.00000000e+00  6.00000000e+00 ...  4.00000000e+00
  -2.94053072e+04  3.23458380e+04]]


In [44]:
model.fit(X_train_reshaped, Y_train, epochs=50, batch_size=32, validation_data=(X_test_reshaped, Y_test))

# Prediction
predictions = model.predict(X_test_reshaped)

Epoch 1/50


2023-08-16 00:53:58.139717: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:58.441731: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:58.555576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:58.649281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:58.793544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - ETA: 0s - loss: 2.3486

2023-08-16 00:53:59.662845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:59.770492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:53:59.819016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 3s 67ms/step - loss: 2.3486 - val_loss: 1.6005
Epoch 2/50
17/17 [==============================] - 0s 22ms/step - loss: 1.9610 - val_loss: 1.5944
Epoch 3/50
17/17 [==============================] - 0s 21ms/step - loss: 1.9415 - val_loss: 1.5863
Epoch 4/50
17/17 [==============================] - 0s 21ms/step - loss: 1.9223 - val_loss: 1.5923
Epoch 5/50
17/17 [==============================] - 0s 21ms/step - loss: 1.9288 - val_loss: 1.5832
Epoch 6/50
17/17 [==============================] - 0s 21ms/step - loss: 1.8910 - val_loss: 1.5909
Epoch 7/50
17/17 [==============================] - 0s 21ms/step - loss: 1.8788 - val_loss: 1.5731
Epoch 8/50
17/17 [==============================] - 0s 21ms/step - loss: 1.9113 - val_loss: 1.6699
Epoch 9/50
17/17 [==============================] - 0s 24ms/step - loss: 1.8844 - val_loss: 1.5637
Epoch 10/50
17/17 [==============================] - 0s 23ms/step - loss: 1.8542 - val_loss: 1.5770
Epoch 11/50
17/17 [=

2023-08-16 00:54:18.954181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:54:19.052149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-16 00:54:19.101479: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 16ms/step
